In [40]:
import requests
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd

def get_ids():
    ids = []
    with open("quest_idx.txt", "r") as file: #this file is from questie addon to get all quest ids
        quest_idx = file.readlines()
        for i,x in enumerate(quest_idx):
            if i<48: continue
            try: 
                match = re.search(r'\[(\d+)\]', x)
                ids.append(match.group(1))
            except AttributeError: break
    return ids

def write_quest_description(path, ids):
    with open(path, 'w', newline='') as file:
        writer = csv.writer(file)  
        writer.writerow(["id", "description"])   
        for j,id in enumerate(ids):
            url = "https://classicdb.ch/?quest=" + id
            response = requests.get(url)

            soup = BeautifulSoup(response.content, "html.parser")

            description_header = soup.find("h3", string="Description")
            if description_header is None:
                continue
            quest_description = ""

            for sibling in description_header.next_siblings:
                if sibling.name == "h3":
                    break
                if sibling.string:
                    quest_description += sibling.string.strip()
            
            writer.writerow([id, quest_description])

def remove_duplicates(source_path, target_path=None):
    # Read in the CSV file
    df = pd.read_csv(source_path)
    def extract_string(s):
        return s[:20]

    df.iloc[:, 1] = df.iloc[:, 1].apply(extract_string)
    df = df.drop_duplicates(subset=df.columns[1], keep='first')


    # Save the new CSV file
    if target_path:
        df.to_csv(target_path, index=False)
        
